<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/05_TS_archs_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: timeseriesAI@gmail.com

# Import libraries 📚

In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
# stable = True # Set to True for latest pip version or False for main branch in GitHub
# !pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [7]:
from tsai.all import *
# from tsai.custom import total_params
my_setup()

os              : Linux-4.18.0-348.23.1.el8_5.x86_64-x86_64-with-glibc2.17
python          : 3.8.13
tsai            : 0.3.2
fastai          : 2.7.7
fastcore        : 1.5.11
torch           : 1.11.0
device          : cpu
cpu cores       : 16
threads per cpu : 1
RAM             : 251.62 GB
GPU memory      : N/A


In [13]:
from IPython.display import clear_output


# Experiments 🧪

I've run a small test to show how you can build any model in the `tsai` library using the `create_model` functions.

The esiest way to do it is pass the architecture you want to use, a `TSDataLoaders` and any `kwargs` you'd like to use. The `create_model` function will pick up the necessary data from the `TSDataLoaders` object.

I've used 2 multivariate time series datasets. As you can see, it's difficult to predict which architecture will work best for which dataset.

Please, bear in mind that this is a very simple test without any hyperparameter tuning.

In [15]:
dsid = 'NATOPS' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, total_params(model), vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
    clear_output()
    display(results)

clear_output()

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,ResCNN,{},268551,0.040261,0.106178,0.983333,19
1,FCN,{},285446,0.073328,0.115541,0.977778,18
2,ResNet,{},490758,0.028025,0.206430,0.933333,32
3,xresnet1d34,{},7232518,0.023036,0.296338,0.916667,90
4,LSTM,"{'n_layers': 1, 'bidirectional': False}",51006,0.384695,0.345031,0.838889,28
5,LSTM,"{'n_layers': 2, 'bidirectional': False}",131806,0.330229,0.384896,0.827778,58
6,LSTM,"{'n_layers': 1, 'bidirectional': True}",102006,0.441388,0.398297,0.816667,48
7,LSTM,"{'n_layers': 3, 'bidirectional': False}",212606,0.964975,0.931788,0.500000,85


LSTM


epoch,train_loss,valid_loss,accuracy,time
0,1.795510,1.792027,0.166667,00:00
1,1.792918,1.791841,0.166667,00:00
2,1.791832,1.791627,0.166667,00:00
3,1.791649,1.791357,0.166667,00:00
4,1.791266,1.791030,0.166667,00:01
5,1.791335,1.790592,0.166667,00:01
6,1.791016,1.790041,0.166667,00:00
7,1.791114,1.789367,0.166667,00:01
8,1.790820,1.788552,0.172222,00:00
9,1.790516,1.787456,0.177778,00:01


KeyboardInterrupt: 

In [14]:
clear_output()

In [6]:
from tsai.custom import total_params
total_params

<function tsai.custom.total_params(model)>

In [ ]:
dsid = 'LSST' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, total_params(model)[0], vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,XceptionTime,{},401580,0.074442,1.488087,0.678832,100
1,LSTM,"{'n_layers': 3, 'bidirectional': True}",572414,0.000234,2.650948,0.669505,98
2,ResCNN,{},260367,0.250915,1.266159,0.667883,60
3,LSTM_FCN,{'shuffle': False},314950,0.440930,1.236977,0.657340,59
4,ResNet,{},482574,0.030630,2.362593,0.650446,78
5,LSTM,"{'n_layers': 2, 'bidirectional': False}",125414,0.002532,2.354824,0.650041,56
6,LSTM,"{'n_layers': 2, 'bidirectional': True}",330814,0.000203,2.637241,0.644769,76
7,LSTM,"{'n_layers': 3, 'bidirectional': False}",206214,0.002217,2.461025,0.633820,66
8,LSTM,"{'n_layers': 1, 'bidirectional': False}",44614,0.054411,1.819723,0.616788,45
9,LSTM,"{'n_layers': 1, 'bidirectional': True}",89214,0.053924,1.760913,0.596918,56
